In [1]:
# load data
import numpy as np
import pandas as pd
from keras.layers import Embedding, LSTM, Dense, Flatten, Dropout
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from sklearn.metrics import log_loss
import gc

train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")

# replace
# train_df['text'] = train_df['text'].str.replace('[^a-zA-Z0-9]', ' ')
# test_df['text'] =test_df['text'].str.replace('[^a-zA-Z0-9]', ' ')



Using TensorFlow backend.


In [3]:
def get_lstm_feats(a=20000,b=10,c=300,bat=32):
    # return train pred prob and test pred prob 
    NUM_WORDS = a
    N = b
    MAX_LEN = c
    NUM_CLASSES = 3
    MODEL_P = '/tmp/lstm.h5'
    
    X = train_df['text']
    Y = train_df['author']
    X_test = test_df['text']

    tokenizer = Tokenizer(num_words=NUM_WORDS)
    tokenizer.fit_on_texts(X)

    train_x = tokenizer.texts_to_sequences(X)
    train_x = pad_sequences(train_x, maxlen=MAX_LEN)
    
    test_x = tokenizer.texts_to_sequences(X_test)
    test_x = pad_sequences(test_x, maxlen=MAX_LEN)

    lb = preprocessing.LabelBinarizer()
    lb.fit(Y)

    train_y = lb.transform(Y)
    
    model = Sequential()
    model.add(Embedding(NUM_WORDS, N, input_length=MAX_LEN))
    model.add(LSTM(N, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1024))
    model.add(Dropout(0.2))
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.summary()
    
    model_chk = ModelCheckpoint(filepath=MODEL_P, monitor='val_loss', save_best_only=True, verbose=1)
    np.random.seed(42)
    model.fit(train_x, train_y, 
              validation_split=0.1,
              batch_size=bat, epochs=5, 
              verbose=2,
              callbacks=[model_chk],
              shuffle=False
             )
    
#     model = load_model(MODEL_P)
#     train_pred = model.predict(train_x)
#     test_pred = model.predict(test_x)
    del model
    gc.collect()
#     print(log_loss(train_y,train_pred))
#     return train_pred,test_pred

print('def lstm done')

def lstm done


In [5]:
for a in [8000, 12000,16000,20000,24000,28000,32000]:
    for b in [12]:
        for c in [300]:
            print(a,b,c)
            get_lstm_feats(a,b,c,256)
            print('----------------------')

8000 12 300
Train on 17621 samples, validate on 1958 samples
Epoch 1/5
Epoch 00001: val_loss improved from inf to 0.72420, saving model to /tmp/lstm.h5
 - 26s - loss: 0.9873 - acc: 0.5076 - val_loss: 0.7242 - val_acc: 0.7007
Epoch 2/5
Epoch 00002: val_loss improved from 0.72420 to 0.48216, saving model to /tmp/lstm.h5
 - 25s - loss: 0.5579 - acc: 0.7772 - val_loss: 0.4822 - val_acc: 0.7988
Epoch 3/5
Epoch 00003: val_loss improved from 0.48216 to 0.45165, saving model to /tmp/lstm.h5
 - 25s - loss: 0.3661 - acc: 0.8595 - val_loss: 0.4516 - val_acc: 0.8166
Epoch 4/5
Epoch 00004: val_loss did not improve
 - 26s - loss: 0.2788 - acc: 0.8936 - val_loss: 0.4679 - val_acc: 0.8238
Epoch 5/5
Epoch 00005: val_loss did not improve
 - 25s - loss: 0.2254 - acc: 0.9138 - val_loss: 0.5126 - val_acc: 0.8253
----------------------
12000 12 300
Train on 17621 samples, validate on 1958 samples
Epoch 1/5
Epoch 00001: val_loss improved from inf to 0.80862, saving model to /tmp/lstm.h5
 - 25s - loss: 1.0170

KeyboardInterrupt: 